<div>
<img src="NGCreativeWorks_horizontal.png" width="400"/>
</div>

## The Tenzing Wireframe

#### Mission Statement
This product is designed to assist creative writing professionals in their efforts to craft narratives for branded content that feel authentic to the National Geographic brand. The aim of this application is to provide creatives with a unique tool that understands the creative writing process in the context of an ad sales environment. It should be able to flexibly take input from users and provide suggestions where necessary, thereby facilitating the development of narrative storytelling without forcing the user into a box.

#### Critical Prep Work

In [1]:
# load dependencies
import os
import pandas as pd
import random
import openai

# turn off warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
from dotenv import load_dotenv
load_dotenv()

openai.organization = "National Geographic"
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [3]:
openai.Model.list()

AuthenticationError: No such organization: National Geographic.

In [7]:
%env

{'__CFBundleIdentifier': 'com.apple.Terminal',
 'TMPDIR': '/var/folders/m6/5m5yr4vx1hnfwqqw70nydyqr0000gn/T/',
 'XPC_FLAGS': '0x0',
 'TERM': 'xterm-color',
 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.9fFLsvi595/Listeners',
 'XPC_SERVICE_NAME': '0',
 'TERM_PROGRAM': 'Apple_Terminal',
 'TERM_PROGRAM_VERSION': '447',
 'TERM_SESSION_ID': '5FA74607-436C-4EE1-BF52-B5FB97946DFE',
 'SHELL': '/bin/zsh',
 'HOME': '/Users/davidlevy',
 'LOGNAME': 'davidlevy',
 'USER': 'davidlevy',
 'PATH': '/Users/davidlevy/Desktop/google-cloud-sdk/bin:/opt/anaconda3/bin:/opt/anaconda3/condabin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin',
 'SHLVL': '1',
 'PWD': '/Users/davidlevy/Desktop/Professional/Nat Geo/Tenzing/tenzing9000/gpt-api-creative-writing-assistant',
 'OLDPWD': '/Users/davidlevy/Desktop/Professional/Nat Geo/Tenzing/tenzing9000',
 'CONDA_EXE': '/opt/anaconda3/bin/conda',
 '_CE_M': '',
 '_CE_CONDA': '',
 'CONDA_PYTHON_EXE': '/opt/anaconda3/bin/python',
 'CONDA_SHLVL

In [125]:
# prepare the OpenAI APIconnection
openai.api_key = "" #insert openai key

# set the creativity parameter to 0.8 globally based on personal experience
temp = 0.8

In [126]:
# load the priority talent roster dataset
filepath = "talent_resources/priority_roster_012323.csv"
talent_df = pd.read_csv(filepath,encoding="ISO-8859-1")

# clean the name fields by removing `?` chracters
talent_df["First Name"] = talent_df["First Name"].str.replace("?", "")
talent_df["Last Name"] = talent_df["Last Name"].str.replace("?", "")

# clean the Professions and Photography Skills fields by removing semi-colons
talent_df["Professions"] = talent_df["Professions"].str.replace(";", ", ")
talent_df["Photography Skills"] = talent_df["Photography Skills"].str.replace(";", ", ")

# add a full name column
talent_df["Full Name"] = talent_df["First Name"] + " " + talent_df["Last Name"]

# visualize the dataframe
talent_df.head(4)

,First Name,Last Name,National Geographic Titles,Residential Country,Professions,Specialities,Photography Skills,Brand Preferences,Brand Affiliation,Mini Bio,Hobbies,Charity Interests,Instagram,Website,Photography Genres,Full Name
0,Lynsey,Addario,Explorer & Photographer,United States,Photographer,NaN,NaN,NaN,NaN,Lynsey Addario is a photojournalist based in L...,NaN,NaN,https://www.instagram.com/lynseyaddario/,https://www.lynseyaddario.com,NaN,Lynsey Addario
1,Lujan,Agusti,Explorer & Photographer,NaN,Photographer,NaN,NaN,NaN,NaN,"Puerto Madryn, Argentina, 1986. National Geogr...",NaN,NaN,https://www.instagram.com/lujanag/?hl=en,https://www.lujanagusti.com/,NaN,Lujan Agusti
2,Sue,Aikens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.instagram.com/sueaikens,https://www.kavikrivercamp.com/,NaN,Sue Aikens
3,Diva,Amon,Explorer,Trinidad and Tobago,"Conservationist, Ecologist, Marine Biologist...",Especially open-ocean and deep-sea biology,NaN,Anything ocean-related or environment-related,No,Dr. Diva Amon is a Caribbean marine biologist ...,"I'm a complete water baby - scuba diving, snor...",SpeSeas (Trinbagonian management and usage of ...,https://www.instagram.com/diva_amon/,https://divaamon.com/,NaN,Diva Amon


In [127]:
# run a quick test to ensure we can comfortably grab random talent as needed
random.sample(sorted(talent_df["Full Name"]), 10)

['Prasenjeet Yadav',
 'Bethany Mollenkof',
 'John Stanmeyer',
 'Kiliii Yüyan',
 'Chris Hemsworth',
 'Bertie Gregory',
 'Jan Pol',
 'Lujan Agusti',
 'Will Smith',
 'Jared Soares']

For demo purposes, we begin with five critical components of any story that will eventually become video content on National Geographic platforms:

* **Protagonist**
    * Who is our hero and what do we know about them?

* **Action**
    * What is our protagonist doing?

* **Quest**
    * What is our protagonist trying to achieve, and how does their journey echo the themes of the story?

* **Location**
    * Where in the world is our protagonist, and how does that location support the story of their journey?

* **Tools**
    * What tools does our protagonist need in order to help them achieve their goal? How do those tools aid the protagonist on their path to victory?

In [112]:
# what ideas about a protagonist might the user bring with them to the table?
protagonist_options = {1 : "Generic Explorer",
                       2 : "Generic Photographer",
                       3 : "Specific talent",
                       4 : "I need suggestions"}

# action should be a fairly free-form input, so we'll leave it more open-ended
action_options = {1 : "I have some action in mind",
                  2 : "I need suggestions"}

# the quest can come from the brief, it could be an idea in the writer's head,
# or it could come from the talent profile
quest_options = {1 : "I have a quest idea to guide the protagonist",
                 2 : "I need suggestions"}

# location can also be somewhat open-ended, but might rely on the action and the quest
location_options = {1 : "I have a location in mind",
                    2 : "I need suggestions"}

# tools are either coming specifically from the brief or they could be fitted into the story ad hoc
tool_options = {1 : "I am writing to a brief and have a specific brand/product to incorporate",
                2 : "This is proactive, so it would be great to see some product integration suggestions"}

The general ideas is that at each step, we'll use the information we have to retrieve the best possible output from the OpenAI API. To make that happen, we need to split all of our steps into discrete functions that can handle user input and understand where in the writing process we are. This is the most complicated piece, and it will likely take a large number of iterations to get it right. But we have to start somewhere!

In [113]:
# a function that takes user input and handles 
def RetrieveProtagonist(user_input):
    
    if user_input == "1":
        
        profession_options = {1 : "I have an idea",
                              2 : "It's not important",
                              3 : "I need suggestions"}
        
        print("\n\nGreat, we'll just reference a generic Nat Geo Explorer.", 
              "\nDo you have an idea of what this Explorer's profession is?\n")
        
        for opt in profession_options:
            print("(" + str(opt) + ")" + " " + profession_options[opt])
            
        prof_pick = input("\nSelect an option:  ")
        
        profession = RetrieveProfession(prof_pick)
        protagonist = "National Geographic Explorer" + profession
        
    elif user_input == "2":
        specialty_options = {1 : "I have an idea",
                             2 : "It's not important",
                             3 : "I need suggestions"}
        
        print("\n\nGreat, we'll just reference a generic Nat Geo Photographer.", 
              "\nDo you have an idea of what this Photographer's specialty is?\n")
        
        for opt in profession_options:
            print("(" + str(opt) + ")" + " " + profession_options[opt])
            
        spec_pick = input("\nSelect an option: ")
        
        specialty = RetrieveSpecialty(spec_pick)
        specialty = "whose work with " + specialty # + short AI-generated phrase about audience impact
        
    elif user_input == "3":
        pass
        
    elif user_input == "4":
        pass
        
    else:
        return False
    
    return protagonist

In [114]:
# a function that returns text containing profession information based on user input passed as an argument
def RetrieveProfession(user_input):
    
    if user_input == "1":
        
        prof = input("\n\nWonderful, what's their job?  ")
        # code to ask GPT to turn this into a job title
        prof = " and " + prof
    
    elif user_input == "2":
        
        prof = ""
    
    elif user_input == "3":
        # logic to print out a few random ideas from the profession list
        prof = " and marine biologist"
    
    else:
        return False
    
    return prof

In [115]:
# a function that returns a generic woman's, man's or unisex name based on user input
def RetrieveGender():
    
    gender = input("Is gender critical to the story?\n\n" + 
                   "(1) Female \n(2) Male \n(3) It doesn't matter" +
                  "\n\nSelect one:  ")
    
    if gender == "1":
        name = ", Jane Doe"
        
    elif gender == "2":
        name = ", John Doe"
        
    else:
        name = ", Bryce Doe"
    
    return name

In [116]:
def WriteStory():
    
    # initiate protagonist sequence
    
    print("Tell me about the hero of your story: \n")
    
    for opt in protagonist_options:
        print("(" + str(opt) + ")" + " " + protagonist_options[opt])
        
    prot_pick = input("\nSelect an option from the menu above:  ")
    
    protagonist = RetrieveProtagonist(prot_pick)
    name = RetrieveGender()
    
    print(f"\n\nOkay, then. We're writing a story about {protagonist + name}." +
          "\n\nWhat do you want to tackle now?\n\n" + 
          "(1) The location \n(2) The quest \n(3) The tools \n(4) The lessons learned")
    
    next_move = input("\n\nSelect one:  ")
    
    return 

#### The Demo

In [119]:
WriteStory()

Tell me about the hero of your story: 

(1) Generic Explorer
(2) Generic Photographer
(3) Specific talent
(4) I need suggestions

Select an option from the menu above:  1


Great, we'll just reference a generic Nat Geo Explorer. 
Do you have an idea of what this Explorer's profession is?

(1) I have an idea
(2) It's not important
(3) I need suggestions

Select an option:  1


Wonderful, what's their job?  cave diver
Is gender critical to the story?

(1) Female 
(2) Male 
(3) It doesn't matter

Select one:  1\


Okay, then. We're writing a story about National Geographic Explorer and cave diver, Bryce Doe.

What do you want to tackle now?

(1) The location 
(2) The quest 
(3) The tools 
(4) The lessons learned


KeyboardInterrupt: Interrupted by user

#### The MVP

#### The Future of Tenzing

In [ ]:
response = openai.Completion.create(model="text-davinci-002",
                                    temperature=creativity, 
                                    max_tokens=2049,
                                    prompt=generate_prompt(creative_product, creativity, brand, product, genre))